# A statistical analysis of droplets generated in ensembles of randomly corrugated ligaments

We study the statistical description of droplet sizes, created as a result
of capillary-induced breakup of ligaments. **Direct numerical simulations** of *air-water systems*
are employed by solving the two-phase Navier-Stokes equations on adaptive Octree grids
[Basilisk](http://basilisk.fr/), using the VOF methodology coupled with height function based
curvature modeling. Breakup of individual ligaments are triggered by initial surface
corrugations, the dynamics of which are deterministic. **Stochasticity** is introduced in the mix
by conducting an *ensemble of simulations* of slender corrugated ligaments, each realization
corresponding to a **random but unique initial configuration**. Probability density functions of
the droplet sizes are computed for different ensemble sizes. Thus, by combining
the effects of stochasticity with the capillarity-driven non-linear dynamics, we can improve our understanding of the nature of drop size distributions encountered in realistic
and convoluted fluid fragmentation scenarios.

> #### TL;DR 
> - corrugated water ligaments in air (~ 100,000's) 
> - ligaments breakup due to complicated effects 
> - ensembles of drops generated (~1000,000's) 
> - probability distributions of drops?   


## Problem Setup 
We have an air-water configuration, thus for the given material properties the length scale of our problem is governed by 

$$ \textrm{Oh}= \frac{\mu}{\sqrt{\rho \sigma R}} \quad , \quad \Lambda = L/W \,.$$
> The reference density and viscosity correspond to the liquid. 

As we do not incorporate any axial stretching rate or initial velocity distribution inside the ligament, our setup can be understood to be at the limit of $\textrm{We} \to 1$ . 

### Computational Domain
The ligament is placed in a square domain of size $L$, with a mean radius $R$. The radial profile along the axial direction is given by 

$$ R(x) = R + \epsilon(x) \quad , \, \text{where} \quad  \epsilon \sim \mathcal{N}(0,\lambda_0^2) $$

The schematic of the computational setup is shown below -

<img src="./schematic.png" style="width: 500px;" align="center" />

### Surface generation
We deal with slender ligaments (i.e. length >> diameter), and we impose periodicity along the axial direction. The surface (radial profile) of this axisymmetric ligament is generated by superposing waves of different wavelenghts, the amplitudes of which are **normally distributed**. This newly generated surface is passed through a **low-pass filter**, which allows us to smoothen the profile and circumvent the extremely sharp peaks/kinks on the surface (our solvers don't like discontinuous derivatives). An example of the surface generation and subsequent filtering is shown below -

<img src="./profile.png" style="width: 600px;" align="center"/>


Thus, we can clearly see that by filtering out all but the $n_c$ longest discrete waves, the final shape of the ligament can be characterized by 

$$ K = n_c \cdot (2\pi W/L) \quad , \quad \varepsilon = \lambda / R \,. $$

> $\lambda_0$ corresponds to the un-filtered surface, while $\lambda$ corresponds to the filtered one. 

#### Energy supplied to discrete modes 
Contrary to the continuous problem, in our numerical simulations we can only inject energy into a **discrete** set of wavenumbers, as demonstrated below using the dispersion relation for the canonical Rayleigh-Plateau instability. 

<img src="waves.png" style="width: 600px;" align="center" />

Thus, the total number of wavenumbers perturbed in the unstable part of the spectrum is 

$$ \Delta k = \Lambda/\pi - 1 $$

Thus, instead of the infinite number of continuous wavenumbers in the unstable part of the spectrum interacting amongst themselves, we get only a handful of them in our discrete setup. At a later part of the analysis, we will see if this any meaningful impact on the size distributions of the drops formed.

### Destabilization of an individual ligament

Let us rescale our physical time $t$ using the capillary timescale at the length scale $R$ of the ligaments. Thus, our time variable is now given by 

$$ T = t \cdot \left(\rho R^3 / \sigma \right)^{-1/2} $$

As an example, we choose a millimeter scale ligament, whose surface is *weakly perturbed*. It should be noted that **all** of the individual ligaments in this particular **ensemble** are characterized by $\Phi$, which is defined as 

$$ \Phi = \left(\textrm{Oh} = 10^{-2}, \varepsilon \approx 0.04 , K = 2\pi , \Lambda = 50 \right) \,.$$

> $\Phi$ acts as a point in the parameter space of all possible corrugated ligament configurations. 

The shape of the ligament initially evolves as per the exponential growth phase well described by linear perturbation theory. Once the perturbations attain a certain amplitude, the non-linear growth kicks in, given by the energy tranfers between different frequencies. The animation below shows the temporal evolution of the ligament, which eventually breaks up into drops, which themselves might undergo coalescence with neighbouring drops. 

![animation](./ligament_breakup.gif)

The colormap on the top half represents the axial velocity, whereas the bottom half corresponds to vorticity. 

## Data analysis 

We have datasets corresponding to variation in the following parameters of the ligament ensemble
- Strongly perturbed and weakly perturbed : $\varepsilon = \{0.038, 0.076\}$
- Smaller and larger aspect ratios : $\Lambda = \{50,100\}$

We start by loading the required libraries, and set the plotting styles. 

In [9]:
# Required libraries for data arrays, data manipulation, plotting etc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
plt.rcParams['text.latex.preamble']=[r"\usepackage{lmodern}",r'\boldmath']
sns.set(style="white", palette="muted")
sns.set_context("paper")


All of the datasets are concatenated onto a single, structured, ASCII format data file. 

In [10]:
drops = pd.read_csv("results", sep=" ")

In [11]:
drops.head()

,Ohnesorge,aspect-ratio,cut-off,amplitude,time,tag,position,velocity,area,diameter,mass,separation,cells
0,0.01,50,0.5,0.1,0,1,49.4770,0.000000,100.8100,11.33220,761.60000,100.0000,7797
1,0.01,50,0.5,0.1,6,1,49.4352,-0.000002,103.4560,11.48000,791.78400,100.0000,828
2,0.01,50,0.5,0.1,12,1,49.2898,0.000295,101.9160,11.39430,774.17600,100.0000,2487
3,0.01,50,0.5,0.1,18,13,93.7334,-0.081582,10.3715,3.63485,25.13270,27.0941,190
4,0.01,50,0.5,0.1,18,12,84.5200,-0.878589,1.8992,1.55543,1.96938,9.2134,181
